In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import scipy
from scipy.spatial.distance import cdist
import sklearn
from sklearn.cluster import KMeans 
from sklearn import metrics
from sklearn.ensemble import IsolationForest

In [13]:
f = glob.glob("/Users/advaitiyer/Desktop/Sambhavi/SOM/*.csv")[0]
df = pd.read_csv(f, encoding='utf-8',sep=',')
df.head(10)

,Country/Territory,Edition,A1,A2,A3,B1,B2,B3,B4,C1,...,G2,G3,G4,Density (persons per sq. km.),Area (sq. km.),Population,Gdp,V2Peedueq Osp,V2X Feduni,V2Xdd Dd
0,Afghanistan,2019,1,1,1,2,2,1,2,1,...,1,0,1,54.9,652230,35780458,1.910135e+10,0.819,0.055,0.000
1,Albania,2019,3,3,2,3,4,3,3,3,...,2,2,2,111.9,27398,3066126,1.527808e+10,2.585,0.469,0.000
2,Algeria,2019,1,1,1,1,1,1,1,1,...,2,2,1,17.8,2381740,42325923,1.700000e+11,3.029,0.054,0.000
3,Andorra,2019,4,4,4,4,4,4,3,4,...,4,3,4,183.1,468,85684,3.154058e+09,NaN,NaN,NaN
4,Angola,2019,0,2,1,2,1,1,2,1,...,1,1,0,25.2,1246700,31427266,9.463542e+10,0.794,0.000,0.000
5,Antigua and Barbuda,2019,4,4,4,3,4,3,3,3,...,3,3,3,219.2,443,97031,1.727759e+09,NaN,NaN,NaN
6,Argentina,2019,4,4,3,4,4,4,3,3,...,3,4,3,16.5,2736690,45089492,4.500000e+11,2.762,0.940,0.000
7,Armenia,2019,2,2,2,3,3,2,2,1,...,2,2,2,107.4,28203,3030288,1.367280e+10,3.100,0.409,0.000
8,Australia,2019,4,4,4,4,4,4,4,4,...,4,4,3,3.3,7682300,25113443,1.390000e+12,3.200,0.986,0.000
9,Austria,2019,4,4,4,4,4,4,3,4,...,4,4,3,107.1,82445,8828456,4.460000e+11,3.636,0.972,0.153


In [14]:
df = df.dropna()

In [15]:
df1 = df.drop(columns='Country/Territory')
df2 = df.drop(columns='Edition')

In [25]:
grouped_country = df2.groupby("Country/Territory")
grouped_year = df1.groupby("Edition")
def anomaly(groups):
    # out = pd.DataFrame()
    ls = []
    for group, vals in groups:
        data = groups.get_group(group)
        data = data.iloc[:,1:]
        to_model_columns = data.columns[1:33]
        clf = IsolationForest(n_estimators=100,max_samples='auto',bootstrap=False,n_jobs=10)
        clf.fit(data[to_model_columns])
        pred = clf.predict(data[to_model_columns])
        data['anomaly'] = pred
        outliers = data.loc[data['anomaly']==-1]
        outlier_index = list(outliers.index)
        data['col'] = pd.Series([group for i in range(len(data))])
        data = data.fillna(value = group)
        # print(outlier_index)
        ls.append(data)
    return ls

In [26]:
by_country = anomaly(grouped_country)
by_year = anomaly(grouped_year)

  " be removed in 0.22.", DeprecationWarning)
/Users/advaitiyer/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/advaitiyer/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/advaitiyer/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/advaitiyer/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/advaitiyer/opt/anaconda

In [29]:
anomaly_by_country = pd.concat(by_country)
anomaly_by_year = pd.concat(by_year)

In [32]:
country_based = pd.merge(anomaly_by_country, df['Edition'], left_index=True, right_index=True, how='outer')
year_based = pd.merge(anomaly_by_year, df['Country/Territory'], left_index=True, right_index=True, how='outer')

In [35]:
country_based.to_csv('country_based.csv',sep=',',encoding='utf-8')

In [36]:
year_based.to_csv('year_based.csv',sep=',',encoding='utf-8')

In [41]:
year_based1.iloc[:,:32]

,A1,A2,A3,B1,B2,B3,B4,C1,C2,C3,...,G2,G3,G4,Density (persons per sq. km.),Area (sq. km.),Population,Gdp,V2Peedueq Osp,V2X Feduni,V2Xdd Dd
1,3,3,2,3,4,3,3,3,2,2,...,2,2,2,111.9,27398,3066126,1.527808e+10,2.585,0.469,0.000
2,1,1,1,1,1,1,1,1,1,1,...,2,2,1,17.8,2381740,42325923,1.700000e+11,3.029,0.054,0.000
4,0,2,1,2,1,1,2,1,1,0,...,1,1,0,25.2,1246700,31427266,9.463542e+10,0.794,0.000,0.000
6,4,4,3,4,4,4,3,3,2,3,...,3,4,3,16.5,2736690,45089492,4.500000e+11,2.762,0.940,0.000
7,2,2,2,3,3,2,2,1,1,2,...,2,2,2,107.4,28203,3030288,1.367280e+10,3.100,0.409,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,3,3,3,4,4,4,3,4,2,2,...,3,2,2,21.5,12189,261565,8.017876e+08,2.649,0.404,0.013
1383,2,2,1,2,2,1,3,1,0,1,...,1,3,2,33.9,882050,29923119,3.710000e+11,1.235,0.748,0.000
1387,1,0,2,2,1,0,1,1,1,0,...,1,1,1,48.0,527968,25338458,4.041524e+10,1.038,0.000,0.000
1388,3,3,3,2,4,3,3,3,2,2,...,2,1,2,19.1,743398,14222233,2.804551e+10,1.270,0.403,0.000


In [38]:
country_based1 = country_based.loc[country_based['anomaly']==1]
year_based1 = year_based.loc[year_based['anomaly']==1]

,A1,A2,A3,B1,B2,B3,B4,C1,C2,C3,...,Density (persons per sq. km.),Area (sq. km.),Population,Gdp,V2Peedueq Osp,V2X Feduni,V2Xdd Dd,anomaly,col,Country/Territory
1,3,3,2,3,4,3,3,3,2,2,...,111.9,27398,3066126,1.527808e+10,2.585,0.469,0.000,1,2019.0,Albania
2,1,1,1,1,1,1,1,1,1,1,...,17.8,2381740,42325923,1.700000e+11,3.029,0.054,0.000,1,2019.0,Algeria
4,0,2,1,2,1,1,2,1,1,0,...,25.2,1246700,31427266,9.463542e+10,0.794,0.000,0.000,1,2019.0,Angola
6,4,4,3,4,4,4,3,3,2,3,...,16.5,2736690,45089492,4.500000e+11,2.762,0.940,0.000,1,2019.0,Argentina
7,2,2,2,3,3,2,2,1,1,2,...,107.4,28203,3030288,1.367280e+10,3.100,0.409,0.000,1,2019.0,Armenia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,3,3,3,4,4,4,3,4,2,2,...,21.5,12189,261565,8.017876e+08,2.649,0.404,0.013,1,2013.0,Vanuatu
1383,2,2,1,2,2,1,3,1,0,1,...,33.9,882050,29923119,3.710000e+11,1.235,0.748,0.000,1,2013.0,Venezuela
1387,1,0,2,2,1,0,1,1,1,0,...,48.0,527968,25338458,4.041524e+10,1.038,0.000,0.000,1,2013.0,Yemen
1388,3,3,3,2,4,3,3,3,2,2,...,19.1,743398,14222233,2.804551e+10,1.270,0.403,0.000,1,2013.0,Zambia


In [5]:
distortions1 = [] 
inertias1 = [] 
mapping1_country = {} 
mapping2_country = {} 
K = range(1,10)
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(country_based1.iloc[,:32]) 
    kmeanModel.fit(country_based1.iloc[,:32])     
      
    distortions1.append(sum(np.min(cdist(country_based1.iloc[,:32], kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / country_based1.iloc[,:32].shape[0]) 
    inertias1.append(kmeanModel.inertia_)
  
    mapping1_country[k] = sum(np.min(cdist(country_based1.iloc[,:32], kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / country_based1.iloc[,:32].shape[0] 
    mapping2_country[k] = kmeanModel.inertia_

plt.plot(K, distortions1, 'bx-') 
plt.xlabel('Values of K')
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show()

plt.plot(K, inertias1, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show()

In [10]:
km = KMeans(n_clusters=3, random_state=1)
km.fit(country_based1.iloc[,:32])
predict=km.predict(country_based1.iloc[,:32])
country_based1.iloc[,:32]['cluster'] = pd.Series(predict, index=country_based1.iloc[,:32].index)

In [11]:
X = X.join(df['Country/Territory'])

In [12]:
list(X.columns)

['A1',
 'A2',
 'A3',
 'B1',
 'B2',
 'B3',
 'B4',
 'C1',
 'C2',
 'C3',
 'D1',
 'D2',
 'D3',
 'D4',
 'E1',
 'E2',
 'E3',
 'F1',
 'F2',
 'F3',
 'F4',
 'G1',
 'G2',
 'G3',
 'G4',
 'Density (persons per sq. km.)',
 'Area (sq. km.)',
 'Population',
 'Gdp',
 'V2Peedueq Osp',
 'V2X Feduni',
 'V2Xdd Dd',
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 'cluster',
 'Country/Territory']

In [14]:
X = X[['Country/Territory',2013,2014,2015,2016,2017,2018,2019,'A1','A2','A3','B1','B2','B3','B4','C1','C2','C3','D1','D2','D3','D4','E1','E2','E3','F1','F2','F3','F4','G1','G2','G3','G4','Density (persons per sq. km.)',
 'Area (sq. km.)',
 'Population',
 'Gdp',
 'V2Peedueq Osp',
 'V2X Feduni',
 'V2Xdd Dd','cluster']]